<a href="https://colab.research.google.com/github/sugengdcahyo/agent-rag/blob/main/02-reasoning-engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploying agent to reasoning engine

This notebook covers how we can restructure the agent code and ultimately deploy the agent to google vertex ai.

## Required Software Installed Locally

* Python version 3.9, 3.10, or 3.11. **Python3.12 will not work**.

* If you are using VSCode, please install Jupyter Notebook extensions.

* Jupyter notebook. Please follow this [installation guide](https://docs.jupyter.org/en/stable/install.html). You may choose whether you want to install classic jupyter notebook or jupyterlab (the next-gen web ui for jupyter)

    * [Classic jupyter notebook installation guide](https://docs.jupyter.org/en/stable/install/notebook-classic.html)

    * [Jupyterlab installation guide](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)

* Google Cloud CLI. Please follow this [installation guide](https://cloud.google.com/sdk/docs/install-sdk)

### Installing dependencies

In [ ]:
%%writefile requirements.txt

google-cloud-aiplatform
google-cloud-aiplatform[langchain]
google-cloud-aiplatform[reasoningengine]
langchain
langchain_core
langchain_community
langchain-google-vertexai==2.0.8
cloudpickle
pydantic==2.9.2
langchain-google-community
google-cloud-discoveryengine
nest-asyncio
asyncio==3.4.3
asyncpg==0.29.0
cloud-sql-python-connector[asyncpg]
langchain-google-cloud-sql-pg
numpy
pandas
pgvector
psycopg2-binary
langchain-openai
langgraph
traceloop-sdk
opentelemetry-instrumentation-google-generativeai
opentelemetry-instrumentation-langchain
opentelemetry-instrumentation-vertexai
python-dotenv

Overwriting requirements.txt


In [ ]:
!pip install --upgrade -r requirements.txt

  Using cached numpy-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached cloud_sql_python_connector-1.17.0-py3-none-any.whl.metadata (30 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
INFO: pip is looking at multiple versions of cloud-sql-python-connector[asyncpg] to determine which version is compatible with other requirements. This could take a while.
  Using cached cloud_sql_python_connector-1.16.0-py2.py3-none-any.whl.metadata (30 kB)
  Using cached cloud_sql_python_connector-1.15.0-py2.py3-none-any.whl.metadata (30 kB)
  Using cached cloud_sql_python_connector-1.14.0-py2.py3-none-any.whl.metadata (28 kB)


## Setting up Google Cloud Account

#### Recommended account setup

if you are running this in jupyter notebook locally, you may need to login to google cloud by running the following command from terminal:

```
gcloud auth login
gcloud auth application-default login
```

If you are using Google Colabs, you need to authenticate with your google account by running the following notebook cell.

> Please remember that you will need to do this on each jupyter notebook during this workshop

In [ ]:
# #@markdown ###Authenticate your Google Cloud Account and enable APIs.
# # Authenticate gcloud.
from google.colab import auth
auth.authenticate_user()

In [72]:
!gcloud projects list

PROJECT_ID                  NAME              PROJECT_NUMBER
agent-rag-451702            agent-rag-451702  644240883738
gen-lang-client-0521448746  Gemini API        672065512482
mekarsa                     mekarsa           1006687429524
windsight                   windsight         155402211297


In [73]:
import requests
from typing import Optional, List
from IPython.display import display, Markdown

from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.tools.retriever import create_retriever_tool
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from langchain.memory import ChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_google_cloud_sql_pg import PostgresChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

from langchain_google_cloud_sql_pg import (
    PostgresEngine,
    PostgresVectorStore,
)

from vertexai.preview import reasoning_engines
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

In [150]:
project_id = "agent-rag-451702"  # @param {type:"string"}
region = "us-central1" #change this to project location
# staging_bucket_name = "devfest24-demo-bucket"  # @param {type:"string"}
instance_name="mkrs-demo" # @param {type:"string"}
database_password = 'testing' # @param {type:"string"}
database_name = 'testing' # @param {type:"string"}
database_user = 'testing' # @param {type:"string"}

assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"

# dont update variable below
!gcloud config set project {project_id} --quiet

# get the ip address of the cloudsql instance
ip_addresses = !gcloud sql instances describe {instance_name} --format="value(ipAddresses[0].ipAddress)"
database_host = ip_addresses[0]

gemini_embedding_model = "text-embedding-004"
gemini_llm_model = "gemini-2.0-flash-001"
embeddings_table_name = "course_content_embeddings"
chat_history_table_name = "chat_histories"

# print(f"API Base URL: {api_base_url}")
print(f"Database Host: {database_host}")

Updated property [core/project].
Database Host: 35.239.247.104


### Initializing vertex ai

In [151]:
import vertexai
vertexai.init(project=project_id, location=region)

### Agent Code

This section below define a few classes that we have tried from the previous notebook.

The main focus here should be `CourseAgent` where we setup all dependencies required by the agent

In [152]:
import psycopg2
import json


class CourseAPIClient:
    def __init__(self):
        """
        db_config: dictionary dengan keys: host, port, dbname, user, password
        """
        db_config = {
            "host": "lecture-dss-db.mekarsa.com",
            "port": 54321,
            "dbname": "warehouse",
            "user": "lecture2024",
            "password": "Ojolali123"
        }
        self.conn = psycopg2.connect(**db_config)
        self.cur = self.conn.cursor()

    def list_tax(self):
        """Mengambil daftar kursus dari database"""
        query = """SELECT
          fpd.kode_rekening,
          fpd.target_murni,
          fpd.totrealisasi_after,
          fpd.persentarget_all,
          dr.nama_rek
        FROM fact_pendapatan_daerah fpd
        join dim_rekening dr
        on fpd.kode_rekening = dr.kode_rekening;"""
        self.cur.execute(query)
        courses = self.cur.fetchall()
        self.close_connection()

        result = [{
            "fpd.kode_rekening": c[0],
            "fpd.terget_realita": float(c[1]),
            "fpd.totrealisasi_after": float(c[2]),
            "fpd.persetarget_all": float(c[3]),
            "dr.nama_rek": c[4]
          } for c in courses]
        return json.dumps(result)

    def get_tax(self, param):
        """Mengambil daftar kursus dari database"""
        query = f"""
        SELECT
          fpd.kode_rekening,
          fpd.target_murni,
          fpd.totrealisasi_after,
          fpd.persentarget_all,
          dr.nama_rek
        FROM
          fact_pendapatan_daerah fpd
        JOIN
          dim_rekening dr
        ON
          fpd.kode_rekening = dr.kode_rekening
        WHERE
          fpd.kode_rekening = '{param}' or dr.nama_rek like '%{param}%';"""
        self.cur.execute(query)
        courses = self.cur.fetchall()
        self.close_connection()

        result = [{
            "fpd.kode_rekening": c[0],
            "fpd.terget_realita": float(c[1]),
            "fpd.totrealisasi_after": float(c[2]),
            "fpd.persentarget_all": float(c[3]),
            "dr.nama_rek": c[4]
          } for c in courses]
        return json.dumps(result)

    def data_report(self, param):
      return "https://api.mekarsa.com/report/report-id"

    def ticket_summaries(self, param):
        """Mengambil daftar kursus dari database"""
        query = f"""
        SELECT
          sum(target_murni) as target,
          sum(totrealisasi_after) as realisasi,
          sum(persentarget_all) as percentage
        FROM fact_pendapatan_daerah fpd
        JOIN dim_rekening dr
        ON fpd.kode_rekening = dr.kode_rekening
        WHERE
          fpd.kode_rekening = '{param}' or dr.nama_rek like '%{param}%';"""
        self.cur.execute(query)
        courses = self.cur.fetchall()
        self.close_connection()

        result = [{
            "target": c[0],
            "realisasi": float(c[1]),
            "percentage": float(c[2])
          } for c in courses]
        return json.dumps(result)

    def close_connection(self):
        """Menutup koneksi ke database"""
        self.cur.close()
        self.conn.close()


In [104]:
api = CourseAPIClient()

In [105]:
api.list_tax()

'[{"fpd.kode_rekening": "4", "fpd.terget_realita": 4271590000000.0, "fpd.totrealisasi_after": 2871066126375.62, "fpd.persetarget_all": 67.2130547729445, "dr.nama_rek": "Pendapatan Daerah"}, {"fpd.kode_rekening": "4.1", "fpd.terget_realita": 771072268674.0, "fpd.totrealisasi_after": 554413320244.93, "fpd.persetarget_all": 71.90160284176027, "dr.nama_rek": "Pendapatan Asli Daerah"}, {"fpd.kode_rekening": "4.1.1", "fpd.terget_realita": 554860000000.0, "fpd.totrealisasi_after": 372499026684.39, "fpd.persetarget_all": 67.13387641646361, "dr.nama_rek": "Hasil Pajak Daerah"}, {"fpd.kode_rekening": "4.1.1.6", "fpd.terget_realita": 50000000000.0, "fpd.totrealisasi_after": 44641477501.46, "fpd.persetarget_all": 89.28295500291999, "dr.nama_rek": "Pajak Jasa Perhotelan"}, {"fpd.kode_rekening": "4.1.1.6.1", "fpd.terget_realita": 49129000000.0, "fpd.totrealisasi_after": 43028719101.46, "fpd.persetarget_all": 87.58313643969956, "dr.nama_rek": "Pajak Hotel"}, {"fpd.kode_rekening": "4.1.1.6.1.1", "fpd.

In [106]:
api = CourseAPIClient()
api.get_tax("4.1")

'[{"fpd.kode_rekening": "4.1", "fpd.terget_realita": 771072268674.0, "fpd.totrealisasi_after": 554413320244.93, "fpd.persentarget_all": 71.90160284176027, "dr.nama_rek": "Pendapatan Asli Daerah"}]'

In [153]:

class GetCourseInput(BaseModel):
    course: str = Field(description="name of the course. this is the unique identifier of the course. it typically contains the course title with dashes, all in lowercase.")

class GetOrderInput(BaseModel):
    order_number: str = Field(description="order number identifier. this is a unique identifier in uuid format.")

class CreateOrderInput(BaseModel):
    course: str = Field(description="name of the course. this is the unique identifier of the course. it typically contains the course title with dashes, all in lowercase.")
    user_name: str = Field(description="name of the user who is purchasing the course .")
    user_email: str = Field(description="email of the user who is purchasing the course.")

class CourseAgent(reasoning_engines.Queryable):
    def __init__(
        self,
        model: str,
        project: str,
        region: str,
        instance: str,
        database: str,
        table: str,
        user: Optional[str] = None,
        password: Optional[str] = None,

    ):
        self.model_name = model
        self.project = project
        self.region = region
        self.instance = instance
        self.database = database
        self.table = table
        self.user = user
        self.password = password
        self.store = {}
        self.agent = None
        self.retriever = None
        self.engine = None

    def __getstate__(self):
        """Custom method for pickling the object."""
        state = self.__dict__.copy()
        # Remove the unpicklable entries
        del state['agent']
        del state['retriever']
        del state['engine']
        return state

    def __setstate__(self, state):
        """Custom method for unpickling the object."""
        self.__dict__.update(state)
        self.agent = None
        self.retriever = None
        self.engine = None
        # Note: set_up() will need to be called after unpickling


    def list_tax(self) -> List[str]:
        """List all available courses sold on the platform."""
        client = CourseAPIClient()
        return client.list_tax()

    def get_course(self, param: str) -> str:
        """Get course details by course name. course name is the unique identifier of the course. it typically contains the course title with dashes.
        This function can be used to get course details such as course price, etc."""
        client = CourseAPIClient()
        return client.get_tax(param)

    def ticket_summaries(self, param: str) -> str:
        """Get course details by course name. course name is the unique identifier of the course. it typically contains the course title with dashes.
        This function can be used to get course details such as course price, etc."""
        client = CourseAPIClient()
        return client.ticket_summaries(param)

    def data_report(self, param: str) -> str:
        """Get course details by course name. course name is the unique identifier of the course. it typically contains the course title with dashes.
        This function can be used to get course details such as course price, etc."""
        client = CourseAPIClient()
        return client.data_report(param)

    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        return PostgresChatMessageHistory.create_sync(
            self.engine,
            table_name=chat_history_table_name,
            session_id=session_id,
        )

    def set_up(self):
        """All unpickle-able logic should go here.
        In general, add any logic that requires a network or database
        connection.
        """

        #  we must initialize the Vertex AI client to use the right project and location
        # otherwise, when the agent is initialized in reasoning engine, it will use reasoning engine project
        vertexai.init(project=self.project, location=self.region)

        # Initialize the vector store
        self.engine = PostgresEngine.from_instance(
            self.project,
            self.region,
            self.instance,
            self.database,
            user=self.user,
            password=self.password,
            quota_project=self.project,
        )

        embeddings_service = VertexAIEmbeddings(model_name=gemini_embedding_model,
                                                project=self.project)

        vector_store = PostgresVectorStore.create_sync(
            self.engine,
            table_name=self.table,
            embedding_service=embeddings_service,
        )
        self.retriever = vector_store.as_retriever(search_kwargs={"k": 10})

        get_course = StructuredTool.from_function(
            func=self.get_course,
            name="get_course",
            description="Jelaskan rincian dari pendapatan atau pajak daerah.",
        )

        list_tax = StructuredTool.from_function(
            func=self.list_tax,
            name="list_tax",
            description="Daftar pendapatan daerah yang terkumpul.",
        )

        ticket_summaries = StructuredTool.from_function(
            func=self.ticket_summaries,
            name="ticket_summaries",
            description="Hitung jumlah pendapatan pada kelompok tiket tertentu."
        )

        data_report = StructuredTool.from_function(
            func=self.data_report,
            name="data_report",
            description="Buat file laporan."
        )

        tools = [get_course, list_tax]

        # Initialize the LLM and prompt
        prompt = {
            "chat_history": lambda x: x["history"],
            "input": lambda x: x["input"],
            "agent_scratchpad": (
                lambda x: format_to_openai_function_messages(x["intermediate_steps"])
            ),
        } | ChatPromptTemplate(
            messages = [
                SystemMessagePromptTemplate.from_template("""
                You are a bot assistant that sells online course about software security. You only use information provided from datastore or tools. You can provide the information that is relevant to the user's question or the summary of the content. If they ask about the content, you can give them more detail about the content. If the user seems interested, you may suggest the user to enroll in the course.
                """),
                MessagesPlaceholder(variable_name="chat_history", optional=True),
                HumanMessagePromptTemplate.from_template("Use tools to answer this questions: {input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )

        safety_settings = {
            HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        }

        ## Model parameters
        model_kwargs = {
            "temperature": 0.5,
            "safety_settings": safety_settings,
        }

        self.agent = reasoning_engines.LangchainAgent(
            model=self.model_name,
            tools=tools,
            prompt=prompt,
            chat_history=self.get_session_history,
            agent_executor_kwargs={
                "return_intermediate_steps": True,
            },
            model_kwargs=model_kwargs,
            enable_tracing=True,
        )
        print("agent is configured")


    def query(self, input: str, session_id: str) -> str:
        """Query the application.

        Args:
            input: The user query.
            session_id: The user's session id.

        Returns:
            The LLM response dictionary.
        """

        response = self.agent.query(
            input=input,
            config={"configurable": {"session_id": session_id}},
        )
        return response

### Instantiating the agent

In [154]:
agent = CourseAgent(
    model=gemini_llm_model,
    project=project_id,
    region=region,
    instance=instance_name,
    database=database_name,
    table=embeddings_table_name,
    user=database_user,
    password=database_password,
)
agent.set_up()

agent is configured


### Testing agent (again) locally

In [155]:
import uuid

# Generate a UUID for the session ID
session_id = str(uuid.uuid4())
print(f"Generated session ID: {session_id}")

Generated session ID: 09953a7f-bef3-4d73-830a-644d8d34c30e


In [156]:
res = agent.query(
    input="data pendapatan apa saja yang kamu simpan?",
    session_id=session_id)

display(Markdown(res["output"]))

Tentu, berikut adalah data pendapatan yang disimpan:
- Pendapatan Daerah
- Pendapatan Asli Daerah
- Hasil Pajak Daerah
- Pajak Jasa Perhotelan
- Pajak Hotel
- Hotel Bintang Lima
- Hotel Bintang Empat
- Hotel Bintang Tiga
- Hotel Bintang Dua
- Hotel Bintang Satu
- Hotel Melati Satu
- Pajak Motel
- Pajak Losmen
- Pajak Gubuk Pariwisata
- Pajak Wisma Pariwisata
- Pajak Pesanggrahan
- Pajak Pesanggrahan (Cotege)
- Pajak Rumah Penginapan dan Sejenisnya
- Pajak Rumah Kos dengan Jumlah Kamar Lebih dari 10 (Sepuluh)
- Pajak Hotel (Kost 11+)
- Pajak Hotel (Kost 20+)
- Pajak Makanan dan/atau Minuman
- Pajak Restoran
- Pajak Restoran dan Sejenisnya
- Pajak Rumah Makan dan Sejenisnya
- Pajak Kafetaria dan Sejenisnya
- Pajak Kantin dan Sejenisnya
- Pajak Warung dan Sejenisnya
- Pajak Bar dan Sejenisnya
- Pajak Jasa Boga/Katering dan Sejenisnya
- Pajak Jasa Kesenian dan Hiburan
- Pajak Tontonan Film
- Pajak Pagelaran Kesenian/Musik/Tari/Busana
- Pajak Kontes Kecantikan, Binaraga dan Sejenisnya
- Pajak Pameran
- Pajak Diskotik, Karoke, Klub Malam dan Sejenisnya
- Karaoke
- Klub Malam
- Pajak Sirkus/Akrobat/Sulap
- Pajak Permainan Biliar dan Bowling
- Pajak Pacuan Kuda, Kendaraan Bermotor dan Permainan Ketangkasan
- Balap Kendaraan Bermotor
- Permainan Ketangkasan
- Pajak Panti Pijat, Refleksi, Mandi Uap/Spa dan Pusat Kebugaran (Fitness Center)
- Panti Pijat/Refleksi
- Mandi Uap/Spa
- Spa Dalam Salon Kecantikan
- Pusat Kebugaran/Fitness Center
- Pajak Pertandingan Olahraga
- Pajak Rekreasi Wahana Air
- Pajak Rekreasi Wahana Ekologi
- Pajak Rekreasi Wahana Pendidikan
- Pajak Rekreasi Wahana Budaya
- Pajak Rekreasi Wahana Salju
- Pajak Rekreasi Wahana Permainan
- Pajak Pemancingan
- Pajak Argowisata
- Pajak Kebun Binatang
- Pajak Tempat Olah Raga Dan Kebugaran
- Pajak Reklame
- Pajak Reklame Papan/Billboard/Videotron/Megatron
- Pajak Reklame Papan/Billboard/Videotron/ Megatron
- Pajak Reklame Kain
- Pajak Reklame Melekat/Stiker
- Pajak Reklame Selebaran
- Pajak Reklame Berjalan
- Pajak Reklame Udara
- Pajak Reklame Apung
- Pajak Reklame Suara
- Pajak Reklame Film/Slide
- Pajak Reklame Peragaan
- Pajak Tenaga Listrik
- Pajak Tenaga Listrik Dihasilkan Sendiri
- Pajak Tenaga Listrik Sumber Lain
- 01. Genset
- 02. Industri
- Pajak Parkir
- Pajak Parkir
- Pajak Parkir
- 01. Pusat Perbelanjaan/ Mall
- 02. Lain - lain
- 03. Cuma-Cuma
- Pajak Air Tanah
- Pajak Air Tanah
- Pajak Air Tanah
- Pajak Sarang Burung Walet
- Pajak Sarang Burung Walet
- Pajak Sarang Burung Walet
- Pajak Mineral Bukan Logam dan Batuan
- Pajak Mineral bukan Logam dan Batuan Lainnya
- Pajak Mineral bukan Logam dan Batuan Lainnya
- Pajak Bumi dan Bangunan Perdesaan dan Perkotaan (PBBP2)
- PBB P2
- PBB P2
- Bea Perolehan Hak Atas Tanah dan Bangunan (BPHTB)
- BPHTB - Pemindahan Hak
- BPHTB - Pemindahan Hak
- BPHTB-Pemberian Hak Baru
- BPHTB-Pemberian Hak Baru
- Hasil Retribusi Daerah
- Retribusi Jasa Umum
- Retribusi Pelayanan Kesehatan
- Retribusi Pelayanan Kesehatan di Puskesmas
- Pendapatan BLUD
- 01. RSUD IA. Moies
- 02. PUSKESMAS
- Retribusi Pelayanan Persampahan/ Kebersihan
- Retribusi Pelayanan Persampahan/ Kebersihan
- Retribusi Pelayanan Pemakaman dan Pengabuan Mayat
- Retribusi Pelayanan Penguburan/Pemakaman termasuk Penggalian dan Pengurukan serta Pembakaran/Pengabuan Mayat
- Retribusi Sewa Tempat Pemakaman atau Pembakaran/Pengabuan Mayat
- Retribusi Pelayanan Parkir di Tepi Jalan Umum
- Retribusi Penyediaan Pelayanan Parkir di Tepi Jalan Umum
- Retribusi Pelayanan Pasar
- Retribusi Pelayanan Pasar
- Retribusi Pengujian Kendaraan Bermotor
- Retribusi Pengujian Kendaraan Bermotor
- Retribusi Pengujian Kendaraan Bermotor Diatas Air
- Retribusi Pemeriksaan Alat Pemadam Kebakaran
- Retribusi Pelayanan Pemeriksaan dan/atau Pengujian Alat Pemadam Kebakaran
- Retribusi Pelayanan Pemeriksaan dan/atau Pengujian Alat Penanggulangan Kebakaran
- Retribusi Pelayanan Pemeriksaan dan/atau Pengujian Alat Penyelamatan Jiwa
- Retribusi Penggantian Biaya Cetak Peta
- Retribusi Penyediaan Peta Dasar (Garis)
- Retribusi Penyediaan Peta Foto
- Retribusi Penyediaan Peta Digital
- Retribusi Penyediaan Peta Tematik
- Retribusi Penyediaan Peta Teknis (Struktur)
- Retribusi Penyediaan dan/atau Penyedotan Kakus
- Retribusi Penyediaan dan/atau Penyedotan Kakus
- Retribusi Penyediaan dan/atau Penyedotan Kakus
- Retribusi Pengelolahan Limbah Cair
- Retribusi Rumah Tangga
- Retribusi Perkantoran
- Retribusi Industri
- Retribusi Pelayanan Tera/Tera Ulang
- Retribusi Pelayanan Pengujian Alat-Alat Ukur, Takar, Timbang, dan Perlengkapannya
- Retribusi Pengujian Barang dalam Keadaan Terbungkus
- Retribusi Pelayanan Pendidikan
- Retribusi Pelayanan Penyelenggaraan Pendidikan Teknis
- Retribusi Pelayanan Penyelenggaraan Pelatihan Teknis
- Retribusi Pelayanan Penyelenggaraan Pendidikan dan Pelatihan Teknis
- Retribusi Pengawasan dan Pengendalian Menara Telekomunikasi
- Retribusi Pengawasan dan Pengendalian Menara Telekomunikasi
- Retribusi Jasa Usaha
- Retribusi Pemakaian Kekayaan Daerah
- Retribusi Penyewaan Tanah dan Bangunan
- Retribusi Penyewaan Tanah
- 01.1 Sewa Tanah Milik Pemerintah/BPKAD
- 01.01 Sewa Tanah Milik Pemerintah/DISHUB
- 01.02.Retribusi Pemakaian Tanah Di Atas HPL Milik Pemkot Samarinda
- 01.03. Retribusi sewa kios/petak/los- Dinas Perdagangan
- Retribusi Penyewaan Bangunan
- 01. Gedung Olahraga Segiri
- 02. Sewa Lapangan Parkir Gedung Olahraga Segiri
- 03. Stadion Segiri
- 04. Sirkuit Kalan
- 05. Rumah Dinas
- * Pasar (Pemanfaatan Aset Daerah)
- Pendapatan Zakat
- Retribusi Pemakaian Laboratorium
- Retribusi Pemakaian Ruangan
- 01. Retr. Pemakaian Aset Daerah Berjualan Dalam Pasar (Disdag)
- Retribusi Pemakaian Kendaraan bermotor
- 01. Bus Besar
- 02. Bus Kecil
- Retribusi Pemakaian Alat
- 01. Alat Laboratorium/Pengujian
- 02. Alat Survey/Pengukuran
- 03. Alat Berat
- Retribusi Pasar Grosir dan/atau Perkotaan
- Retribusi Penyediaan Fasilitas Pasar Grosir Berbagai Jenis Barang yang Dikontrakkan
- Retribusi Penyediaan Fasilitas Pasar/Pertokoan yang Dikontrakkan
- Retribusi Tempat Pelelangan
- Retribusi Penyediaan Tempat Pelelangan
- Retribusi Penyediaan Fasilitas Lainnya di Tempat Pelelangan
- 01. Retribusi sandar Kapal (setiap sandar)
- 02. Pas Masuk untuk perorangan ke area kerja PPI (per Tahun)
- 03. Masuk untuk kendaraan ke area TPI (setiap kali masuk)
- 04. Pemakaian Fasilitas PPI untuk penjualan ikan (pendaratan ikan dan SIKPI)
- 05. Pemakaian Lokasi PPI untuk produksi es balok
- 06. Ret. Jasa Ruang Pendingin Ikan di PPI
- Retribusi Terminal
- Retribusi Pelayanan Penyediaan Tempat Parkir untuk Kendaraan Penumpang dan Bus Umum
- Retribusi Pelayanan Penyediaan Tempat Kegiatan Usaha
- Retribusi Pelayanan Penyediaan Fasilitas Lainnya di Lingkungan Terminal
- Retribusi Tempat Parkir Khusus
- Retribusi Pelayanan Tempat Khusus Parkir
- 01. Untuk Parkir Dalam Area Pasar/DISDAG
- 02. Untuk Taman Buah Bebaya/DISHUB
- 03. Untuk Parkir Gor Segiri-dishub
- 04. Untuk Parkir Gor Segiri-dispora
- 05. Untuk Parkir Plaza 21
- 06. Untuk Parkir Pasar-Pasar
- 07. Untuk Parkir RSUD IA MOEIS
- 08. Untuk Parkir Pasar Merdeka
- 09. Untuk Parkir Pasar Sungai Dama
- 10. Untuk Parkir Taman Samarendah
- 11. Untuk Parkir Pasar Baqa
- Retribusi Tempat Penginapan/ Pesanggrahan/Vila
- Retribusi Pelayanan Tempat Penginapan/ Pesanggrahan/Vila
- 01. Mess Perwakilan Samarinda di Jakarta
- Retribusi Rumah Potong Hewan
- Retribusi Pelayanan Rumah Potong Hewan
- 01. Pemeriksaan Kesehatan Hewan di Rumah Pemotongan Hewan
- 02. Pemeriksaan Kesehatan Unggas dan/atau Daging Unggas di RPH
- 03. Pengandangan di RPH
- Pelayanan Kesehatan untuk Hewan Ternak
- Retribusi Pelayanan Kepelabuhanan
- Retribusi Pelayanan Kepelabuhanan
- 01. Laut
- 01.1 Pelayanan Kapal
- 01.2 Pelayanan Barang
- 01.3 Pelayanan Lainnya
- 02. Angkutan Sungai
- 02.1 Tambat Kapal
- 02.2 Tarif Masuk Dermaga
- 02.3 Lain - lain (mck,wc,kios,toko)
- 02.4 Tarif Masuk Dermaga S. Kunjang
- 02.5 Tarif Masuk Dermaga Pasar Pagi
- 02.5 Tarif Masuk Dermaga Harapan Baru
- 02.5 Tarif Masuk Dermaga Samarinda Seberang
- 02.6 Tarif Masuk Dermaga Loa Janan
- Retribusi Tempat Rekreasi dan Olahraga
- Retribusi Pelayanan Tempat Rekreasi dan Olahraga
- Retribusi Penyebrangan di Air
- Retribusi Pelayanan Penyeberangan Orang
- Retribusi Pelayanan Penyeberangan Barang
- Retribusi Penjualan Produksi Usaha Daerah
- Retribusi Penjualan Produksi Hasil Usaha Daerah berupa Bibit atau Benih Tanaman
- Retribusi Penjualan Produksi hasil Usaha Daerah berupa Bibit Ternak
- Retribusi Penjualan Produksi hasil Usaha Daerah berupa Bibit atau Benih Ikan
- Retribusi Penjualan Produksi hasil Usaha Daerah selain Bibit atau Benih Tanaman, Ternak, dan Ikan
- Retribusi Perizinan Tertentu
- Retribusi Izin Mendirikan Bangunan
- Retribusi Pemberian Izin Mendirikan Bangunan
- Retribusi Izin Tempat Penjualan Minuman Beralkohol
- Retribusi Pemberian Izin Tempat Penjualan Minuman Beralkohol
- Retribusi Izin Trayek untuk Menyediakan Pelayanan Angkutan Umum
- Retribusi Izin Trayek untuk Menyediakan Pelayanan Angkutan Umum
- Retribusi Izin Usaha Perikanan
- Retribusi Pemberian Izin Kegiatan Usaha Penangkapan Ikan
- Retribusi Pemberian Izin Kegiatan Usaha Pembudidayaan Ikan
- Retribusi Pengendalian Lalu Lintas
- Retribusi Pengendalian Lalu Lintas Penggunaan Ruas Jalan Tertentu
- Retribusi Pengendalian Lalu Lintas Penggunaan Koridor Tertentu
- Retribusi Pengendalian Lalu Lintas Penggunaan Kawasan Tertentu
- Retribusi Perpanjangan Izin Mempekerjakan Tenaga Kerja Asing (IMTA)
- Retribusi Pemberian Perpanjangan IMTA kepada Pemberi Kerja Tenaga Kerja Asing
- Hasil Pengelolaan Kekayaan Daerah yang Dipisahkan
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMN
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMN
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMN
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD (Lembaga Keuangan)
- Hasil Selisih Lebih Tukar Menukar Jalan dan Jembatan
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD (Lembaga Keuangan)
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD (Lembaga Keuangan)
- 01. Bankaltim
- 02. Bank Perkreditan Rakyat (BPR) Kota Samarinda
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD (Varian Niaga)
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD (Varian Niaga)
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD (Bidang Air Minum)
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada Perusahaan Milik Daerah/BUMD (Bidang Air Minum)
- 01. Perusahaan Daerah Air Minum (PDAM)
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD (Bidang Limbah)
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD (Bidang Limbah)
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD (Bidang Sanitasi)
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD (Bidang Sanitasi)
- Bagian Laba yang dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada Perusahaan Milik Swasta
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada Perusahaan Milik Swasta
- Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada Perusahaan Milik Swasta
- Lain-lain Pendapatan Asli Daerah yang Sah
- Hasil Penjualan BMD yang Tidak Dipisahkan
- Hasil Penjualan Tanah
- Hasil Penjualan Tanah Digunakan untuk mencatat pendapatan dari hasil penjualan atas tanah.
- Hasil Penjualan Peralatan dan Mesin
- Hasil Penjualan Alat Besar
- Hasil Penjualan Alat Angkutan
- Hasil Penjualan Alat Bengkel dan Alat Ukur
- Hasil Penjualan Alat Pertanian
- Hasil Penjualan Alat Kantor dan Rumah Tangga
- Hasil Penjualan Alat Studio, Komunikasi, dan Pemancar
- Hasil Penjualan Alat Kedokteran dan Kesehatan
- Hasil Penjualan Alat Laboratorium
- Hasil Penjualan Komputer
- Hasil Penjualan Alat Eksplorasi
- Hasil Penjualan Alat Pengeboran
- Hasil Penjualan Alat Produksi, Pengolahan, dan Pemurnian
- Hasil Penjualan Alat Bantu Eksplorasi
- Hasil Penjualan Alat Keselamatan Kerja
- Hasil Penjualan Alat Peraga
- Hasil Penjualan Peralatan Proses/Produksi
- Hasil Penjualan Rambu-rambu
- Hasil Penjualan Peralatan Olahraga
- Hasil Penjualan Gedung dan Bangunan
- Hasil Penjualan Gedung dan Bangunan
- 02.Pendapatan Dari Angsuran/ Cicilan Penjualan
- Angsuran/Cicilan Penjualan Rumah Dinas Daerah Golongan III
- 01. Pokok Angsuran / Cicilan Penjualan Rumah Dinas Daerah Gol III
- 02. Denda Angsuran / Cicilan Penjualan Rumah Dinas Daerah Gol III
- Angsuran/Cicilan Penjualan Kendaraan Perorangan Dinas
- 01. Pokok Angsuran/Cicilan Penjualan Kendaraan Perorangan Dinas
- 02. Denda Angsuran/Cicilan Penjualan Kendaraan Perorangan Dinas
- Angsuran/Cicilan Ganti Kerugian Barang Milik Daerah
- 01. Pokok Angsuran/Cicilan Ganti Kerugian Barang Milik Daerah
- 02. Denda Angsuran/Cicilan Ganti Kerugian Barang Milik Daerah
- Angsuran/Cicilan Rumah Susun
- 01. Pokok Angsuran/Cicilan Rumah Susun
- 02. Denda Angsuran/Cicilan Rumah Susun
- Hasil Penjualan Monumen
- Hasil Penjualan Bangunan Menara
- Hasil Penjualan Tugu Titik Kontrol/Pasti
- Hasil Penjualan Jalan, Jaringan, dan Irigasi
- Hasil Penjualan Jalan dan Jembatan
- Hasil Penjualan Jalan dan Jembatan
- Hasil Penjualan Instalasi
- Hasil Penjualan Jaringan
- Hasil Penjualan Aset Tetap Lainnya
- Hasil Penjualan Bahan Perpustakaan
- Hasil Penjualan Barang Bercorak Kesenian/Kebudayaan/Olahraga
- Hasil Penjualan Hewan
- Hasil Penjualan Biota Perairan
- Hasil Penjualan Tanaman
- Hasil Penjualan Barang Koleksi Non Budaya
- Hasil Penjualan Aset Tetap Dalam Renovasi
- Hasil Penjualan Aset Lainnya
- Hasil Penjualan Aset Lainnya
- Hasil Selisih Lebih Tukar Menukar BMD yang Tidak Dipisahkan
- Hasil Selisih Lebih Tukar Menukar Tanah
- Hasil Selisih Lebih Tukar Menukar Tanah
- Hasil Selisih Lebih Tukar Menukar Peralatan dan Mesin
- Hasil Selisih Lebih Tukar Menukar Alat Besar
- Hasil Selisih Lebih Tukar Menukar Alat Angkutan
- Hasil Selisih Lebih Tukar Menukar Alat Bengkel dan Alat Ukur
- Hasil Selisih Lebih Tukar Menukar Alat Pertanian
- Hasil Selisih Lebih Tukar Menukar Alat Pertanian
- Hasil Selisih Lebih Tukar Menukar Alat Studio, Komunikasi, dan Pemancar
- Hasil Selisih Lebih Tukar Menukar Alat Kedokteran dan Kesehatan
- Hasil Selisih Lebih Tukar Menukar Alat Laboratorium
- Hasil Selisih Lebih Tukar Menukar Komputer
- Hasil Selisih Lebih Tukar Menukar Alat Eksplorasi
- Hasil Selisih Lebih Tukar Menukar Alat Pengeboran
- Hasil Selisih Lebih Tukar Menukar Alat Produksi, Pengolahan, dan Pemurnian
- Hasil Selisih Lebih Tukar Menukar Alat Bantu Eksplorasi
- Hasil Selisih Lebih Tukar Menukar Alat Keselamatan Kerja
- Hasil Selisih Lebih Tukar Menukar Alat Peraga
- Hasil Selisih Lebih Tukar Menukar Peralatan Proses/Produksi
- Hasil Selisih Lebih Tukar Menukar RambuRambu
- Hasil Selisih Lebih Tukar Menukar Peralatan Olahraga
- Hasil Selisih Lebih Tukar Menukar Gedung dan Bangunan
- Hasil Selisih Lebih Tukar Menukar Bangunan Gedung
- Hasil Selisih Lebih Tukar Menukar Monumen
- Hasil Selisih Lebih Tukar Menukar Bangunan Menara
- Hasil Selisih Lebih Tukar Menukar Tugu Titik Kontrol/Pasti
- Hasil Selisih Lebih Tukar Menukar Jalan, Jaringan, dan Irigasi
- Hasil Selisih Lebih Tukar Menukar Bangunan Air
- Hasil Selisih Lebih Tukar Menukar Instalasi
- Hasil Selisih Lebih Tukar Menukar Jaringan
- Hasil Selisih Lebih Tukar Menukar Aset Tetap Lainnya
- Hasil Selisih Lebih Tukar Menukar Bahan Perpustakaan
- Hasil Selisih Lebih Tukar Menukar Barang Bercorak Kesenian/Kebudayaan/Olahraga
- Hasil Selisih Lebih Tukar Menukar Barang Bercorak Kesenian/Kebudayaan/Olahraga
- Hasil Selisih Lebih Tukar Menukar Biota Perairan
- Hasil Selisih Lebih Tukar Menukar Tanaman
- Hasil Selisih Lebih Tukar Menukar Barang Koleksi Non Budaya
- Hasil Selisih Lebih Tukar Menukar Aset Tetap Dalam Renovasi
- Hasil Selisih Lebih Tukar Menukar Aset Lainnya
- Hasil Selisih Lebih Tukar Menukar Aset Lainnya-Aset Tidak Berwujud
- Hasil Selisih Lebih Tukar Menukar Aset Lainnya-Aset Lain-Lain
- Hasil Pemanfaatan BMD yang Tidak Dipisahkan
- Hasil Sewa BMD
- Hasil Sewa BMD
- Hasil Kerja Sama Pemanfaatan BMD
- Hasil Kerja Sama Pemanfaatan BMD
- Hasil dari Bangun Guna Serah
- Hasil dari Bangun Guna Serah
- Hasil dari Bangun Serah Guna
- Hasil dari Bangun Serah Guna
- Hasil dari Kerja Sama Penyediaan Infrastruktur
- Hasil dari Kerja Sama Penyediaan Infrastruktur
- Hasil Kerja Sama Daerah
- Hasil Kerja Sama Daerah
- Hasil Kerja Sama Daerah
- 01. PT. Samaco (Mahakam Lampion Garden)
- 03. PT. Pelabuhan Samudera Palaran (PSP)
- 04. PT. Wahana Abadi Tirtateknilka Sejati (WATS)
- 02. PERUSAHAAN PATUNGAN
- a. Pasar Segiri (PT. Yudha Putera Kaltim)
- b. Pasar Sungai Dama (PT. Surya Rizky Reza Jaya Abadi)
- c. Pasar Kedondong (PT. Bahana Surya Sentosa)
- d. Pasar Pagi (PT. Buluh Perindu)
- Jasa Giro
- Jasa Giro pada Kas Daerah
- Jasa Giro pada Kas Daerah
- Jasa Giro pada Kas di Bendahara
- Jasa Giro pada Kas di Bendahara
- Jasa Giro pada Rekening Dana Cadangan
- Jasa Giro pada Rekening Dana Cadangan
- Jasa Giro pada Rekening Dana BOS
- Jasa Giro pada Rekening Dana BOS
- Jasa Giro Dana Kapitasi pada FKTP
- Jasa Giro Dana Kapitasi pada FKTP
- Hasil Pengelolaan Dana Bergulir
- Hasil Pengelolaan Dana Bergulir
- Hasil Pengelolaan Dana Bergulir
- Pendapatan Bunga
- Pendapatan Bunga atas Penempatan Uang Pemerintah Daerah
- Pendapatan Bunga atas Penempatan Uang Pemerintah Daerah
- Pendapatan Zakat, Infaq, Shadaqah, dan Wakaf
- Penerimaan atas Tuntutan Ganti Kerugian Keuangan Daerah
- Tuntutan Ganti Kerugian Daerah terhadap Bendahara
- Tuntutan Ganti Kerugian Daerah terhadap Bendahara
- Tuntutan Ganti Kerugian Daerah terhadap Pegawai Negeri Bukan Bendahara atau Pejabat Lain
- Tuntutan Ganti Kerugian Daerah terhadap Pegawai Negeri Bukan Bendahara atau Pejabat Lain
- Penerimaan Komisi, Potongan, atau Bentuk Lain
- Penerimaan Komisi, Potongan, atau Bentuk Lain
- Penerimaan Komisi, Potongan, atau Bentuk Lain
- 01. Penerimaan Lain-Lain (Lebih Bayar Pendapatan Pajak / Retribusi atau penerimaan PAD Lain-lainnya)
- Penerimaan Keuntungan dari Selisih Nilai Tukar Rupiah terhadap Mata Uang Asing
- Penerimaan Keuntungan dari Selisih Nilai Tukar Rupiah terhadap Mata Uang Asing
- Penerimaan Keuntungan dari Selisih Nilai Tukar Rupiah terhadap Mata Uang Asing
- Pendapatan Denda atas Keterlambatan Pelaksanaan Pekerjaan
- Pendapatan Denda atas Keterlambatan Pelaksanaan Pekerjaan
- Pendapatan Denda atas Keterlambatan Pelaksanaan Pekerjaan
- Pendapatan Denda Pajak Daerah
- Pendapatan Denda Pajak Hotel
- Pendapatan Denda Pajak Hotel
- Pendapatan Denda Pajak Restoran
- Pendapatan Denda Pajak Restoran dan Sejenisnya
- Pendapatan Denda Pajak Hiburan
- Pendapatan Denda Pajak Hiburan dan Sejenisnya
- Pendapatan Denda Pajak Reklame
- Pendapatan Denda Pajak Reklame dan Sejenisnya
- Pendapatan Denda Pajak Tenaga Listrik
- Pendapatan Denda Pajak Tenaga Listrik
- Pendapatan Denda Pajak Parkir
- Pendapatan Denda Pajak Parkir
- Pendapatan Denda Pajak Air Tanah
- Pendapatan Denda Pajak Air Tanah
- Pendapatan Denda Pajak Sarang Burung
- Walet
- Pendapatan Denda Pajak Sarang Burung Walet
- Pendapatan Denda Pajak Mineral Bukan Logam dan Batuan
- Pendapatan Denda Pajak Mineral Bukan Logam dan Batuan
- Pendapatan Denda Pajak Bumi dan Bangunan Perdesaan dan Perkotaan (PBBP2)
- Pendapatan Denda PBBP2
- Pendapatan Denda Bea Perolehan Hak atas Tanah dan Bangunan (BPHTB)
- Pendapatan Denda BPHTB-Pemindahan Hak
- Pendapatan Denda BPHTB-Pemberian Hak Baru
- Pendapatan Denda Retribusi Daerah
- Pendapatan Denda Retribusi Jasa Umum
- Pendapatan Denda Retribusi Jasa Umum
- Pendapatan Denda Retribusi Jasa Usaha
- Pendapatan Denda Retribusi Jasa Usaha
- Pendapatan Denda Retribusi Perizinan Tertentu
- Pendapatan Denda Retribusi Izin Mendirikan Bangunan
- Pendapatan Zakat
- Pendapatan Denda Retribusi Izin Tempat Penjualan Minuman Beralkohol
- Pendapatan Denda Retribusi Izin Tempat Penjualan Minuman Beralkohol
- Pendapatan Denda Retribusi Izin Usaha Perikanan
- Pendapatan Denda Retribusi Pengendalian Lalu Lintas
- Pendapatan Denda Retribusi Perpanjangan Izin Mempekerjakan Tenaga Kerja Asing (IMTA)
- Pendapatan Hasil Eksekusi atas Jaminan
- Hasil Eksekusi atas Jaminan atas Pengadaan Barang/Jasa
- Hasil Eksekusi atas Jaminan atas Pengadaan Barang/Jasa
- Pendapatan dari Pengembalian
- Pendapatan dari Pengembalian Kelebihan Pembayaran Pajak Penghasilan Pasal 21
- Pendapatan dari Pengembalian Kelebihan Pembayaran Pajak Penghasilan Pasal 21
- Pendapatan dari Pengembalian Kelebihan Pembayaran Iuran Simpanan Peserta Tabungan Perumahan Rakyat ASN
- Pendapatan dari Pengembalian Kelebihan Pembayaran Iuran Simpanan Peserta Tabungan Perumahan Rakyat PNS
- Pendapatan dari Pengembalian Kelebihan Pembayaran Iuran Simpanan Peserta Tabungan Perumahan Rakyat PPPK
- Pendapatan dari Pengembalian Kelebihan Pembayaran Gaji dan Tunjangan
- Pendapatan dari Pengembalian Kelebihan Pembayaran Gaji dan Tunjangan
- Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas
- Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Dalam Negeri Perjalanan Dinas Bia
- Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Dalam Negeri Perjalanan Dinas Tetap
- Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Dalam Negeri Perjalanan Dinas Dalam Kot
- Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Dalam Negeri Perjalanan Dinas Paket Meeting Dalam Kot
- Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Dalam Negeri Perjalanan Dinas Paket Meeting Luar Kota
- Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Luar Negeri Perjalanan Dinas Biasa–Luar Neg
- Pendapatan dari Pengembalian Kelebihan Pembayaran Kecelakaan Kerja (JKK)
- Pendapatan dari Pengembalian Kelebihan Pembayaran JKK
- Pendapatan dari Pengembalian Kelebihan Pembayaran Jaminan Kematian (JKM)
- Pendapatan dari Pengembalian Kelebihan Pembayaran JKM
- Pendapatan dari Pengembalian Kelebihan Pembayaran Jaminan Kesehatan Nasional (JKN)
- Pendapatan dari Pengembalian Kelebihan Pembayaran Jaminan Kesehatan Nasional (JKN)
- Pendapatan Denda Pemanfaatan BMD yang tidak Dipisahkan
- Pendapatan Denda Pengakhiran Sewa BMD
- Pendapatan Denda Pengakhiran Sewa BMD
- Pendapatan Denda Hasil dari Kerja Sama Penyediaan Infrastruktur
- Pendapatan Denda Hasil dari Kerja Sama Penyediaan Infrastruktur
- Pendapatan Dana Kapitasi Jaminan Kesehatan Nasional (JKN) pada Fasilitas Kesehatan Tingkat Pertama (FKTP)
- Pendapatan Dana Kapitasi JKN pada FKTP
- Pendapatan Dana Kapitasi JKN pada FKTP
- Pendapatan Hasil Pengelolaan Dana Bergulir
- Pendapatan Hasil Pengelolaan Dana Bergulir
- Pendapatan Hasil Pengelolaan Dana Bergulir
- Pendapatan Berdasarkan Putusan Pengadilan (Inkracht)
- Pendapatan Berdasarkan Putusan Pengadilan (Inkracht)
- Pendapatan Berdasarkan Putusan Pengadilan (Inkracht)
- Pendapatan Denda atas Pelanggaran Peraturan Daerah
- Pendapatan Denda atas Pelanggaran Peraturan Daerah
- Pendapatan Denda atas Pelanggaran Peraturan Daerah
- Pendapatan Infaq
- Pendapatan Shadaqah
- Pendapatan Wakaf
- Pendapatan Transfer
- Pendapatan Transfer Pemerintah Pusat
- Dana Perimbangan
- Dana Transfer Umum-Dana Bagi Hasil (DBH)
- DBH Pajak Bumi dan Bangunan
- 01. PBB Pertambangan (P3)
- 01.1. PBB Non Migas
- 01.2. PBB Migas
- 01.3.PBB Sektor Lainnya
- 01.4. Pemungutan PBB Migas
- 01.5. Pemungutan PBB Non Migas
- 01.6. Pemungutan Sektor Lainnya
- 01.7. Bagi Rata-rata
- 01.8. Pelimpahan PBB
- 01.9.PBB Perkebunan
- 01.10. PBB Kehutanan
- DBH PPh Pasal 21
- DBH PPh Pasal 25 dan Pasal 29/WPOPDN
- DBH Cukai Hasil Tembakau (CHT)
- DBH Sumber Daya Alam (SDA) Minyak Bumi
- DBH Sumber Daya Alam (SDA) Gas Bumi
- DBH Sumber Daya Alam (SDA) Pengusahaan Panas Bumi
- DBH Sumber Daya Alam (SDA) Mineral dan Batubara-Landrent
- Dana Bagi Hasil (DBH) Sumber Daya Alam (SDA) Mineral dan Batubara-Royalty
- DBH Sumber Daya Alam (SDA) Kehutanan Provisi Sumber Daya Hutan (PSDH)
- DBH Sumber Daya Alam (SDA) Kehutanan Iuran Izin Usaha Pemanfaatan Hutan (IIUPH)
- DBH Sumber Daya Alam (SDA) Kehutanan Dana Reboisasi (DR)
- DBH Sumber Daya Alam (SDA) Perikanan
- DBH Lainnya - Sawit
- Dana Transfer Umum-Dana Alokasi Umum (DAU)
- DAU
- DAU yang Tidak Ditentukan Penggunaannya
- DAU Tambahan Dukungan Pendanaan Kelurahan
- DAU Tambahan Dukungan Pendanaan atas Kebijakan Penyetaraan Penghasilan Tetap Kepala Desa dan Perangkat Desa
- DAU Tambahan Dukungan Pendanaan atas Kebijakan Penggajian Pegawai Pemerintah dengan Perjanjian Kerja
- DAU Tambahan Dukungan Pendanaan atas Kebijakan Penggajian Pegawai Pemerintah dengan Perjanjian Kerja
- Bidang Pendidikan
- Bidang Kesehatan
- Bidang Pekerjaan Umum
- Dana Transfer Khusus-Dana Alokasi Khusus (DAK) Fisik
- DAK Fisik-Bidang Pendidikan-Reguler-PAUD
- DAK Fisik-Bidang Pendidikan-Reguler-SD
- DAK Fisik-Bidang Pendidikan-Reguler-SMP
- DAK Fisik-Bidang Pendidikan-Reguler-SMA
- DAK Fisik-Bidang Pendidikan-Reguler-SLB
- DAK Fisik-Bidang Pendidikan-Reguler-SKB
- DAK Fisik-Bidang Pendidikan-Afirmasi-SD
- DAK Fisik-Bidang Pendidikan-Afirmasi-SMP
- DAK Fisik-Bidang Pendidikan-Afirmasi-SMA
- DAK Fisik-Bidang Pendidikan-Penugasan SMK
- DAK Fisik-Bidang Pendidikan-Reguler Perpustakaan Daerah
- DAK Fisik-Bidang Pendidikan-Reguler Gedung Olahraga
- DAK Fisik-Bidang Kesehatan dan KB Reguler-Pelayanan Kesehaan Dasar
- DAK Fisik-Bidang Kesehatan dan KB Reguler-Pelayanan Kesehatan Rujukn
- DAK Fisik-Bidang Kesehatan dan KB Reguler-Pelayanan Kefarmasin
- DAK Fisik-Bidang Kesehatan dan KB Penugasan-Penurunan Angka Kematian Ibu (AKI), Angka Kematian Bayi (AKB) dan Intervesi Stunting
- DAK Fisik-Bidang Kesehatan dan KB Penugasan-Penguatan Intervensi Stunting
- Fisik-Bidang Kesehatan dan KB Penugasan-Peningkatan Pencegahan dan
Pengendalian Penyakit dan Sanitasi Total
Berbasis Masyarakat
- DAK Fisik-Bidang Kesehatan dan KB Penugasan-Penguatan RS Rujukan
Nasional/Provinsi/Regional/Pariwisata
- DAK Fisik-Bidang Kesehatan dan KB Penugasan-Pembangunan RS Pratama
- DAK Fisik-Bidang Kesehatan dan KB Penugasan-Puskesmas Pariwisata
- DAK Fisik-Bidang Kesehatan dan KB Penugasan-Balai Pelatihan Kesehatan
- DAK Fisik-Bidang Kesehatan dan KB Afirmasi-Penguatan Puskesmas Daerah Tertinggal Perbatasan Negara dan Kepulauan (DTPK)
- DAK Fisik-Bidang Kesehatan dan KB Afirmasi-Penguatan Prasarana Dasar Puskesmas
- DAK Fisik-Bidang Kesehatan dan KB Reguler-KB
- DAK Fisik-Bidang Kesehatan dan KB Penugasan-Pen

In [157]:
response = agent.query(
    input="kelompokkan data berdasarkan rekening level 3?",
    session_id=session_id)
display(Markdown(response["output"]))

Tentu, berikut adalah pengelompokan data berdasarkan rekening level 3:

*   **4.1.1 Hasil Pajak Daerah**
*   **4.1.2 Hasil Retribusi Daerah**
*   **4.1.3 Hasil Pengelolaan Kekayaan Daerah yang Dipisahkan**
*   **4.1.4 Lain-lain Pendapatan Asli Daerah yang Sah**
*   **4.2.1 Pendapatan Transfer Pemerintah Pusat**
*   **4.2.2 Pendapatan Transfer Antar Daerah**
*   **4.3.1 Pendapatan Hibah**
*   **4.3.2 Dana Darurat**
*   **4.3.3 Lain-lain Pendapatan Sesuai dengan Ketentuan Peraturan Perundang-Undangan**

In [158]:
response = agent.query(
    input="berapa total pendapatan Hasil Retribusi Daerah?",
    session_id=session_id)
display(Markdown(response["output"]))

Total pendapatan Hasil Retribusi Daerah adalah Rp150.557.778.000,00

In [162]:
response = agent.query(
    input="tampilkan rekening dan nilai angka pada rekening hasil retribusi daerah?",
    session_id=session_id)
display(Markdown(response["output"]))

Berikut adalah rincian rekening dan nilai angka pada rekening Hasil Retribusi Daerah:

*   **Kode Rekening:** 4.1.2
*   **Nama Rekening:** Hasil Retribusi Daerah
*   **Target:** Rp 150.557.778.000,00
*   **Realisasi:** Rp 105.813.856.641,03
*   **Persentase Target:** 70.28%


In [163]:
response = agent.query(
    input="berikan list data lengkap pendapatan 5 saja?",
    session_id=session_id)
display(Markdown(response["output"]))

Tentu, berikut adalah 5 data pendapatan teratas beserta informasi lengkapnya:

1.  **Pendapatan Daerah**

    *   Kode Rekening: 4
    *   Target Realisasi: Rp 4.271.590.000.000,00
    *   Total Realisasi: Rp 2.871.066.126.375,62
    *   Persentase Target: 67.21%
2.  **Pendapatan Asli Daerah**

    *   Kode Rekening: 4.1
    *   Target Realisasi: Rp 771.072.268.674,00
    *   Total Realisasi: Rp 554.413.320.244,93
    *   Persentase Target: 71.90%
3.  **Hasil Pajak Daerah**

    *   Kode Rekening: 4.1.1
    *   Target Realisasi: Rp 554.860.000.000,00
    *   Total Realisasi: Rp 372.499.026.684,39
    *   Persentase Target: 67.13%
4.  **Pajak Jasa Perhotelan**

    *   Kode Rekening: 4.1.1.6
    *   Target Realisasi: Rp 50.000.000.000,00
    *   Total Realisasi: Rp 44.641.477.501,46
    *   Persentase Target: 89.28%
5.  **Pajak Hotel**

    *   Kode Rekening: 4.1.1.6.1
    *   Target Realisasi: Rp 49.129.000.000,00
    *   Total Realisasi: Rp 43.028.719.101,46
    *   Persentase Target: 87.58%



In [164]:
response = agent.query(
    input="rekening mana yang mencapai target tepat 100%?",
    session_id=session_id)
display(Markdown(response["output"]))

Berikut adalah daftar rekening yang mencapai target 100%:

*   **4.1.1.6.3 Pajak Losmen**
*   **4.1.1.6.3.1 Pajak Losmen**
*   **4.1.1.6.6 Pajak Pesanggrahan**
*   **4.1.1.6.6.1 Pajak Pesanggrahan (Cotege)**
*   **4.1.1.6.8 Pajak Rumah Kos dengan Jumlah Kamar Lebih dari 10 (Sepuluh)**
*   **4.1.1.6.8.1 Pajak Rumah Kos dengan Jumlah Kamar Lebih dari 10 (Sepuluh)**
*   **4.1.1.6.8.1.1 Pajak Hotel  (Kost 11+)**
*   **4.1.1.6.8.1.2 Pajak Hotel  (Kost 20+)**
*   **4.1.1.8.11 Pajak Rekreasi Wahana Air**
*   **4.1.1.8.11.1 Pajak Rekreasi Wahana Air**
*   **4.1.1.8.12 Pajak Rekreasi Wahana Ekologi**
*   **4.1.1.8.12.1 Pajak Rekreasi Wahana Ekologi**
*   **4.1.1.8.15 Pajak Rekreasi Wahana Salju**
*   **4.1.1.8.15.1 Pajak Rekreasi Wahana Salju**
*   **4.1.2.1.7 Retribusi Pemeriksaan Alat Pemadam Kebakaran**
*   **4.1.2.1.7.1 Retribusi Pelayanan Pemeriksaan dan/atau Pengujian Alat Pemadam Kebakaran**
*   **4.1.2.1.9 Retribusi Penyediaan dan/atau Penyedotan Kakus**
*   **4.1.2.1.9.1 Retribusi Penyediaan dan/atau Penyedotan Kakus**
*   **4.1.2.1.9.1.3 Retribusi Penyediaan dan/atau Penyedotan Kakus**
*   **4.1.2.1.11 Retribusi Pelayanan Tera/Tera Ulang**
*   **4.1.2.1.11.1 Retribusi Pelayanan Pengujian Alat-Alat Ukur, Takar, Timbang, dan Perlengkapannya**
*   **4.1.2.2.1.2.2 01.01 Sewa Tanah Milik Pemerintah/DISHUB**
*   **4.1.2.2.1.3.6 \* Pasar (Pemanfaatan Aset Daerah)**
*   **4.1.2.2.3.2 Retribusi Penyediaan Fasilitas Lainnya di Tempat Pelelangan**
*   **4.1.2.2.3.2.4 04. Pemakaian Fasilitas PPI untuk penjualan ikan (pendaratan ikan dan SIKPI)**
*   **4.1.2.2.5.1.4 04. Untuk  Parkir Gor Segiri-dispora**
*   **4.1.2.2.5.1.5 05. Untuk Parkir Plaza 21**
*   **4.1.2.2.5.1.6 06. Untuk Parkir Pasar-Pasar**
*   **4.1.2.2.5.1.7 07. Untuk Parkir RSUD IA MOEIS**
*   **4.1.2.2.5.1.8 08. Untuk Parkir Pasar Merdeka**
*   **4.1.2.2.5.1.9 09. Untuk Parkir Pasar Sungai Dama**
*   **4.1.2.2.5.1.10 10. Untuk Parkir Taman Samarendah**
*   **4.1.2.2.7.1.2 02. Pemeriksaan Kesehatan Unggas dan/atau Daging Unggas di RPH**
*   **4.1.2.2.8.1.7 02.2 Tarif Masuk Dermaga**
*   **4.1.2.2.8.1.9 02.4 Tarif Masuk Dermaga S. Kunjang**
*   **4.1.2.2.8.1.10 02.5 Tarif Masuk Dermaga Pasar Pagi**
*   **4.1.2.2.8.1.11 02.5 Tarif Masuk Dermaga Harapan Baru**
*   **4.1.2.2.8.1.12 02.5 Tarif Masuk Dermaga Samarinda Seberang**
*   **4.1.2.2.8.1.13 02.6 Tarif Masuk Dermaga Loa Janan**
*   **4.1.3.2.2.1 Bagian Laba yang Dibagikan kepada Pemerintah Daerah (Dividen) atas Penyertaan Modal pada BUMD (Varian Niaga)**
*   **4.1.4.1.3 Hasil Penjualan Gedung dan Bangunan**
*   **4.1.4.1.3.1 Hasil Penjualan Gedung dan Bangunan**
*   **4.1.4.1.3.1.2 Angsuran/Cicilan Penjualan Rumah Dinas Daerah Golongan III**
*   **4.1.4.5.1 Jasa Giro pada Kas Daerah**
*   **4.1.4.5.1.1 Jasa Giro pada Kas Daerah**
*   **4.1.4.5.2 Jasa Giro pada Kas di Bendahara**
*   **4.1.4.5.2.1 Jasa Giro pada Kas di Bendahara**
*   **4.1.4.7 Pendapatan Bunga**
*   **4.1.4.7.1 Pendapatan Bunga atas Penempatan Uang Pemerintah Daerah**
*   **4.1.4.7.1.1 Pendapatan Bunga atas Penempatan Uang Pemerintah Daerah**
*   **4.1.4.12.16 Pendapatan Denda Bea Perolehan Hak atas Tanah dan Bangunan (BPHTB)**
*   **4.1.4.12.16.2 Pendapatan Denda BPHTB-Pemberian Hak Baru**
*   **4.1.4.13.1 Pendapatan Denda Retribusi Jasa Umum**
*   **4.1.4.13.1.1 Pendapatan Denda Retribusi Jasa Umum**
*   **4.1.4.13.2 Pendapatan Denda Retribusi Jasa Usaha**
*   **4.1.4.13.2.1 Pendapatan Denda Retribusi Jasa Usaha**
*   **4.1.4.15.3 Pendapatan dari Pengembalian Kelebihan Pembayaran Gaji dan Tunjangan**
*   **4.1.4.15.3.1 Pendapatan dari Pengembalian Kelebihan Pembayaran Gaji dan Tunjangan**
*   **4.1.4.15.4 Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas**
*   **4.1.4.15.4.1 Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Dalam Negeri Perjalanan Dinas Bia**
*   **4.1.4.15.4.3 Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Dalam Negeri Perjalanan Dinas Dalam Kot**
*   **4.1.4.15.4.6 Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Luar Negeri Perjalanan Dinas Biasa–Luar Neg**
*   **4.1.4.21 Pendapatan Denda atas Pelanggaran Peraturan Daerah**
*   **4.1.4.21.1 Pendapatan Denda atas Pelanggaran Peraturan Daerah**
*   **4.1.4.21.1.1 Pendapatan Denda atas Pelanggaran Peraturan Daerah**
*   **4.2.1.1.1.4 DBH Cukai Hasil Tembakau (CHT)**
*   **4.2.1.1.1.15 01.4. Pemungutan PBB Migas**
*   **4.2.1.1.3.25 DAK Fisik-Bidang Kesehatan dan KB Reguler-KB**
*   **4.1.4.4.1.1.1 01. PT. Samaco (Mahakam Lampion Garden)**
*   **4.1.4.5.1 Jasa Giro pada Kas Daerah**
*   **4.1.4.5.1.1 Jasa Giro pada Kas Daerah**
*   **4.1.4.5.2 Jasa Giro pada Kas di Bendahara**
*   **4.1.4.5.2.1 Jasa Giro pada Kas di Bendahara**
*   **4.1.4.7 Pendapatan Bunga**
*   **4.1.4.7.1 Pendapatan Bunga atas Penempatan Uang Pemerintah Daerah**
*   **4.1.4.7.1.1 Pendapatan Bunga atas Penempatan Uang Pemerintah Daerah**
*   **4.1.4.12.16 Pendapatan Denda Bea Perolehan Hak atas Tanah dan Bangunan (BPHTB)**
*   **4.1.4.12.16.2 Pendapatan Denda BPHTB-Pemberian Hak Baru**
*   **4.1.4.13.1 Pendapatan Denda Retribusi Jasa Umum**
*   **4.1.4.13.1.1 Pendapatan Denda Retribusi Jasa Umum**
*   **4.1.4.13.2 Pendapatan Denda Retribusi Jasa Usaha**
*   **4.1.4.13.2.1 Pendapatan Denda Retribusi Jasa Usaha**
*   **4.1.4.15.3 Pendapatan dari Pengembalian Kelebihan Pembayaran Gaji dan Tunjangan**
*   **4.1.4.15.3.1 Pendapatan dari Pengembalian Kelebihan Pembayaran Gaji dan Tunjangan**
*   **4.1.4.15.4 Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas**
*   **4.1.4.15.4.1 Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Dalam Negeri Perjalanan Dinas Bia**
*   **4.1.4.15.4.3 Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Dalam Negeri Perjalanan Dinas Dalam Kot**
*   **4.1.4.15.4.6 Pendapatan dari Pengembalian Kelebihan Pembayaran Perjalanan Dinas Luar Negeri Perjalanan Dinas Biasa–Luar Neg**
*   **4.1.4.21 Pendapatan Denda atas Pelanggaran Peraturan Daerah**
*   **4.1.4.21.1 Pendapatan Denda atas Pelanggaran Peraturan Daerah**
*   **4.1.4.21.1.1 Pendapatan Denda atas Pelanggaran Peraturan Daerah**



In [166]:
response = agent.query(
    input="rekening level 2 mana yang mencapai target tepat 60% hingga 70%?",
    session_id=session_id)
display(Markdown(response["output"]))

Berikut adalah rekening level 2 yang mencapai target antara 60% hingga 70%:

*   **4.2 Pendapatan Transfer** dengan persentase target 65.42%


In [167]:
import unittest

class TestCourseAgent(unittest.TestCase):

    def setUp(self):
        # Initialize the agent (replace with your actual agent initialization)
        self.agent = CourseAgent(
            model=gemini_llm_model,
            project=project_id,
            region=region,
            instance=instance_name,
            database=database_name,
            table=embeddings_table_name,
            user=database_user,
            password=database_password,
        )
        self.agent.set_up()
        self.session_id = str(uuid.uuid4())

    def test_list_tax(self):
        response = self.agent.query("data pendapatan apa saja yang kamu simpan?", self.session_id)
        self.assertIn("pendapatan", response["output"].lower()) # Example assertion, adjust as needed

    def test_get_course(self):
      response = self.agent.query("berikan detail data pajak dengan kode rekening 4.1", self.session_id)
      self.assertIn("4.1", response["output"]) # Example assertion, adjust as needed

    def test_ticket_summaries(self):
      response = self.agent.query("berapa total pendapatan Hasil Retribusi Daerah?", self.session_id)
      self.assertIn("total pendapatan", response["output"].lower())  # Example assertion, adjust as needed


    def test_data_report(self):
        response = self.agent.query("buat laporan untuk data pajak", self.session_id)
        self.assertIn("report", response["output"].lower()) # Example assertion, adjust as needed

    def test_invalid_query(self):
        response = self.agent.query("This is gibberish", self.session_id)
        # Assert that the response indicates an invalid query or provides a helpful message.
        self.assertIn("not understand", response["output"].lower())

    # Add more test cases for other functionalities and edge cases
    # For example, test for empty queries, queries with special characters, or queries that require specific tool calls.


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


agent is configured


F

agent is configured


.

agent is configured


F

agent is configured


.

agent is configured


.
FAIL: test_data_report (__main__.TestCourseAgent.test_data_report)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-167-b95d5ef1233e>", line 35, in test_data_report
    self.assertIn("report", response["output"].lower()) # Example assertion, adjust as needed
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: 'report' not found in 'berikut adalah laporan data pajak daerah:\n\ntotal target pendapatan daerah adalah rp 4.271.590.000.000,00 dengan total realisasi sebesar rp 2.871.066.126.375,62 atau 67,21%. pendapatan asli daerah (pad) memiliki target sebesar rp 771.072.268.674,00 dengan realisasi rp 554.413.320.244,93 atau 71,90%.\n\nsecara rinci, hasil pajak daerah dengan kode rekening 4.1.1 memiliki target sebesar rp 554.860.000.000,00 dengan realisasi rp 372.499.026.684,39 (67,13%). pajak jasa perhotelan (4.1.1.6) mencapai 89,28% dari target, sedangkan pajak makanan dan/atau minuman (4

### Deploying the Agent on Vertex AI

Before deploying the agent, we are going to grant excessive permissions for the agent first. Ideally, you should only grant the permissions that are required by the agent. But just to make sure that the agent is able to run without any issues, we are going to grant all the permissions for now. :p

In [137]:
# Retrieve the project number associated with your project ID
from googleapiclient import discovery
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=project_id)
response = request.execute()
project_number = response["projectNumber"]
project_number

'644240883738'

In [138]:
!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/serviceusage.serviceUsageConsumer"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/discoveryengine.editor"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/cloudsql.admin"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/aiplatform.admin"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/aiplatform.user"

Updated IAM policy for project [agent-rag-451702].
bindings:
- members:
  - serviceAccount:service-644240883738@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:644240883738-compute@developer.gserviceaccount.com
  - serviceAccount:644240883738@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-644240883738@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - user:sugengdcahyo@gmail.com
  role: roles/cloudsql.client
- members:
  - serviceAccount:service-644240883738@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-644240883738@containerregistry.iam.gserviceaccount.com
  role: roles/containerregistry.ServiceAgent
- members:
  - serviceAccount:644240883738-compute@developer.gserviceaccount.com
  - serviceAccount:644240883738@cloudservices.gserviceaccount.co

#### Deploying to reasoning engine

Deploying is as simple as calling `create()` method. We will provide the agent here and some dependencies required to run the agent.

In [139]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform",
        "google-cloud-aiplatform[langchain]",
        "google-cloud-aiplatform[reasoningengine]",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain-google-vertexai==2.0.8",
        "cloudpickle",
        "pydantic==2.9.2",
        "langchain-google-community",
        "google-cloud-discoveryengine",
        "nest-asyncio",
        "asyncio==3.4.3",
        "asyncpg==0.29.0",
        "cloud-sql-python-connector[asyncpg]",
        "langchain-google-cloud-sql-pg",
        "numpy",
        "pandas",
        "pgvector",
        "psycopg2-binary",
        "google-cloud-trace"
    ],
    display_name="course-agent",
    sys_version="3.11",

)
remote_agent

# remote_agent = reasoning_engines.ReasoningEngine.create(
#     agent,
#     requirements=[
#         "google-cloud-aiplatform==1.73.0",
#         "google-cloud-aiplatform[langchain]",
#         "google-cloud-aiplatform[reasoningengine]",
#         "langchain==0.3.9",
#         "langchain_core==0.3.21",
#         "langchain_community==0.3.9",
#         "langchain-google-vertexai==2.0.8",
#         "cloudpickle==3.1.0",
#         "pydantic==2.9.2",
#         "langchain-google-community==2.0.3",
#         "google-cloud-discoveryengine==0.13.4",
#         "nest-asyncio==1.6.0",
#         "asyncio==3.4.3",
#         "asyncpg==0.29.0",
#         "cloud-sql-python-connector[asyncpg]==1.13.0",
#         "langchain-google-cloud-sql-pg==0.11.1",
#         "numpy",
#         "pandas",
#         "pgvector",
#         "psycopg2-binary",
#         "google-cloud-trace==1.13.5"
#     ],
#     display_name="course-agent",
#     sys_version="3.11",

# )
# remote_agent

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket devfest24-demo-bucket
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://devfest24-demo-bucket/reasoning_engine/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://devfest24-demo-bucket/reasoning_engine/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://devfest24-demo-bucket/reasoning_engine/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/644240883738/locations/us-central1/reasoningEngines/3489427694095958016/operations/1003159121731518464


KeyboardInterrupt: 

### Testing Remote Agent

In [ ]:
from vertexai.preview import reasoning_engines

reasoning_engines.ReasoningEngine.list()
# remote_agent = reasoning_engines.ReasoningEngine("projects/908311267620/locations/us-central1/reasoningEngines/7151100481752793088")


In [ ]:
import uuid

# Generate a UUID for the session ID
session_id = str(uuid.uuid4())
print(f"Generated session ID: {session_id}")

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Can you please share what are being taught on this course",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Does it teach about how to design a forgot password system securely?",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="How much this course costs?",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Yes. I want to enroll",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Name is Mulyono and email is mulyono@gmail.com",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="I have made the payment. Can you please check?",
  session_id=session_id,
)
display(Markdown(response["output"]))